In [1]:
import nest_asyncio
import os

from llama_index.llms.openai import OpenAI
from llama_index.indices.managed.llama_cloud import LlamaCloudIndex

from IPython.display import display, Markdown

from llama_index.core.tools import QueryEngineTool

In [2]:
nest_asyncio.apply()

In [3]:
index = LlamaCloudIndex(
  name=os.getenv("INDEX_NAME"), 
  project_name="Default",
  organization_id=os.getenv("ORG_ID"),
  api_key=os.getenv("LLAMA_API_KEY")
)

In [4]:
query = "What about DC?"

In [5]:
response = index.as_query_engine().query(query)

In [7]:
response.response

'Washington, D.C. is the capital city and federal district of the United States. It was named after George Washington, the first president of the United States, and is not part of any state. The city was founded in 1791 and serves as the seat of the U.S. federal government. Washington, D.C. is known for its historical significance, hosting various national monuments, museums, and government buildings. Additionally, the city is recognized for its diverse culture, arts scene, cuisine, and as a major political and tourist destination.'

In [ ]:
display(Markdown(response.response))

Washington, D.C. is the capital city and federal district of the United States. It was named after George Washington, the first president of the United States, and is not part of any state. The city was founded in 1791 and serves as the seat of the U.S. federal government. Washington, D.C. is known for its historical significance, hosting various federal government headquarters, national monuments, museums, and international organizations. Additionally, the city is recognized for its diverse culture, arts scene, cuisine, and as a major tourist destination.

## Add context so that you can have a conversation with the bot

In [8]:
llama_cloud_query_engine = index.as_query_engine()

In [9]:
llama_cloud_tool = QueryEngineTool.from_defaults(
    query_engine=llama_cloud_query_engine,
    description=(
        f"Useful for answering semantic questions about certain states in the US."
    ),
    name="llama_cloud_tool"
)

In [ ]:
from llama_index.core.agent.workflow import FunctionAgent
from llama_index.llms.openai import OpenAI
from llama_index.core.workflow import Context

In [12]:
agent = FunctionAgent(
    name="state-guide-usa",
    description= f"Useful for answering semantic questions about certain states in the US.",
    tools=[llama_cloud_tool],
    llm=OpenAI(model="gpt-4o-mini"),
    system_prompt="You are useful for answering semantic questions about certain states in the US.",
)

In [13]:
ctx = Context(agent)

In [14]:
response = await agent.run(
    user_msg="What state has the highest population?", ctx=ctx
)
print(str(response))

California has the highest population of any state in the United States.


In [15]:
response = await agent.run(
    user_msg="What is the capital of that state?", ctx=ctx
)
print(str(response))

The capital of California is Sacramento.
